<a href="https://colab.research.google.com/github/kevinmoreno1-ux/ENTREGAS/blob/main/04-modelo_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Configurar rutas de archivos
train_file_path = '/content/drive/MyDrive/IA/SaberPro/train.csv'
test_file_path = '/content/drive/MyDrive/IA/SaberPro/test.csv'
output_file_path = '/content/drive/MyDrive/IA/SaberPro/submission_knn.csv'

print("=== MODELO K-NEAREST NEIGHBORS (KNN) - VERSIÓN RÁPIDA ===")
print("=" * 70)

# ============================================================================
# CARGA Y PREPROCESAMIENTO
# ============================================================================

print("Cargando datos de entrenamiento...")
try:
    train_data = pd.read_csv(train_file_path)
    print(f"✓ Datos de entrenamiento cargados: {train_data.shape}")
except Exception as e:
    print(f"✗ Error cargando train.csv: {e}")
    raise

# Limpieza inicial
if 'F_TIENEINTERNET.1' in train_data.columns:
    train_data = train_data.drop(columns=["F_TIENEINTERNET.1"])

# Separar características
numeric_cols = train_data.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train_data.select_dtypes(include=['object']).columns.tolist()

# Excluir variable objetivo e ID
for col in ['RENDIMIENTO_GLOBAL', 'ID']:
    if col in numeric_cols:
        numeric_cols.remove(col)
    if col in categorical_cols:
        categorical_cols.remove(col)

print(f"Columnas numéricas: {len(numeric_cols)}")
print(f"Columnas categóricas: {len(categorical_cols)}")

# Manejo de valores faltantes
print("Aplicando preprocesamiento...")
train_data[numeric_cols] = train_data[numeric_cols].fillna(train_data[numeric_cols].median())
train_data[categorical_cols] = train_data[categorical_cols].fillna('Unknown')

# Codificación de variables categóricas
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col].astype(str))
    label_encoders[col] = le

# Preparar datos para entrenamiento
X = train_data.drop(columns=["RENDIMIENTO_GLOBAL", "ID"])
y = train_data["RENDIMIENTO_GLOBAL"]

# Verificar distribución de clases
class_distribution = y.value_counts().sort_index()
print(f"\nDistribución de clases:")
for cls, count in class_distribution.items():
    print(f"  Clase {cls}: {count} muestras ({count/len(y)*100:.1f}%)")

# ============================================================================
# PREPROCESAMIENTO ESPECIAL PARA KNN
# ============================================================================

print("\nAplicando escalado para KNN...")
# KNN es muy sensible al escalado, así que escalamos todas las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("✓ Datos escalados para KNN")

# Dividir para evaluación rápida (opcional, puedes eliminar esto para más velocidad)
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)
print(f"✓ Conjunto de entrenamiento: {X_train.shape}")
print(f"✓ Conjunto de validación: {X_val.shape}")

# ============================================================================
# MODELO KNN CON VALOR FIJO DE K
# ============================================================================

print("\nEntrenando modelo KNN con k=5...")

# Modelo KNN con valor fijo (k=5 es generalmente un buen valor por defecto)
model = KNeighborsClassifier(
    n_neighbors=5,        # Valor fijo - puedes cambiarlo si quieres probar otros
    weights='distance',   # 'uniform' o 'distance' (pesa por distancia)
    algorithm='auto',     # 'auto', 'ball_tree', 'kd_tree', 'brute'
    leaf_size=30,
    p=2,                  # Distancia euclidiana (p=2)
    metric='minkowski',
    n_jobs=-1             # Usar todos los cores
)

# Entrenar con todos los datos (más rápido que con división train/val)
model.fit(X_scaled, y)

print("✓ Modelo KNN entrenado exitosamente!")

# Evaluación rápida (opcional - comentar si quieres ahorrar más tiempo)
try:
    val_score = model.score(X_val, y_val)
    print(f"✓ Precisión en validación: {val_score:.4f}")
except:
    print("✓ Evaluación de validación omitida para mayor velocidad")

# ============================================================================
# PROCESAMIENTO DE DATOS DE PRUEBA
# ============================================================================

print("\nCargando datos de prueba...")
try:
    test_data = pd.read_csv(test_file_path)
    print(f"✓ Datos de prueba cargados: {test_data.shape}")
except Exception as e:
    print(f"✗ Error cargando test.csv: {e}")
    raise

# Conservar IDs
test_ids = test_data['ID'].copy()

# Limpieza inicial
if 'F_TIENEINTERNET.1' in test_data.columns:
    test_data = test_data.drop(columns=["F_TIENEINTERNET.1"])

# Asegurar que las columnas coincidan
missing_cols = set(X.columns) - set(test_data.columns)
if missing_cols:
    print(f"⚠ Columnas faltantes en test: {missing_cols}")
    for col in missing_cols:
        test_data[col] = 0

extra_cols = set(test_data.columns) - set(X.columns) - {'ID'}
if extra_cols:
    print(f"⚠ Columnas adicionales en test: {extra_cols}")
    test_data = test_data.drop(columns=extra_cols)

test_data = test_data[X.columns.tolist() + ['ID']]

# Aplicar mismo preprocesamiento
test_data[numeric_cols] = test_data[numeric_cols].fillna(train_data[numeric_cols].median())

# Codificar variables categóricas con los mismos encoders
for col in categorical_cols:
    if col in test_data.columns and col in label_encoders:
        test_data[col] = test_data[col].astype(str)
        # Manejar valores no vistos
        unknown_mask = ~test_data[col].isin(label_encoders[col].classes_)
        if unknown_mask.any():
            # Reemplazar con el valor más frecuente
            most_frequent = label_encoders[col].classes_[0]
            test_data.loc[unknown_mask, col] = most_frequent
        test_data[col] = label_encoders[col].transform(test_data[col])

# Preparar datos de prueba
X_test = test_data.drop(columns=['ID'])
X_test_scaled = scaler.transform(X_test)

# ============================================================================
# PREDICCIÓN Y ANÁLISIS
# ============================================================================

print("Realizando predicciones...")
y_pred = model.predict(X_test_scaled)

# Análisis de las predicciones
unique_preds, counts = np.unique(y_pred, return_counts=True)
print(f"\nDistribución de predicciones:")
for pred, count in zip(unique_preds, counts):
    percentage = count/len(y_pred)*100
    print(f"  Clase {pred}: {count:6d} muestras ({percentage:5.1f}%)")

# ============================================================================
# GUARDAR RESULTADOS
# ============================================================================

# Crear archivo de submission
submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': y_pred
})

submission.to_csv(output_file_path, index=False)
print(f"\n✓ Archivo de submission guardado en: {output_file_path}")
print(f"✓ Total de predicciones: {len(y_pred)}")

# Información adicional del modelo
print(f"\nConfiguración del modelo KNN:")
print(f"  Número de vecinos (k): 5 (fijo)")
print(f"  Métrica de distancia: {model.metric}")
print(f"  Peso: {model.weights}")

print("=" * 70)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
=== MODELO K-NEAREST NEIGHBORS (KNN) - VERSIÓN RÁPIDA ===
Cargando datos de entrenamiento...
✓ Datos de entrenamiento cargados: (692500, 21)
Columnas numéricas: 5
Columnas categóricas: 13
Aplicando preprocesamiento...

Distribución de clases:
  Clase alto: 175619 muestras (25.4%)
  Clase bajo: 172987 muestras (25.0%)
  Clase medio-alto: 171619 muestras (24.8%)
  Clase medio-bajo: 172275 muestras (24.9%)

Aplicando escalado para KNN...
✓ Datos escalados para KNN
✓ Conjunto de entrenamiento: (554000, 18)
✓ Conjunto de validación: (138500, 18)

Entrenando modelo KNN con k=5...
✓ Modelo KNN entrenado exitosamente!
✓ Precisión en validación: 1.0000

Cargando datos de prueba...
✓ Datos de prueba cargados: (296786, 20)
Realizando predicciones...
